# Phase 1 Notebook: `graph.py`

Target file: `execution/langgraph/graph.py`

Purpose: implement LangGraph node orchestration with schema validation, policy checks, duplicate-tool protection, and checkpointing.

## 3-Layer Context

- Layer 1 (Directive): this is the P1 orchestration centerpiece requested by `directives/phase1_langgraph.md`.
- Layer 2 (Orchestration): defines node graph (`plan -> execute -> policy -> finalize`).
- Layer 3 (Execution): node handlers call deterministic tools/checkpoint stores and only use LLM for planning output.

In [ ]:
from pathlib import Path
import sys
import importlib.util

def bootstrap_repo_root() -> Path:
    cwd = Path.cwd().resolve()
    candidates = [cwd, *cwd.parents, Path('/home/nir/dev/agent_phase0')]
    for candidate in candidates:
        if (candidate / 'execution' / 'langgraph' / 'graph.py').exists():
            if str(candidate) not in sys.path:
                sys.path.insert(0, str(candidate))
            return candidate
    raise RuntimeError('Could not locate repo root for graph notebook.')

repo_root = bootstrap_repo_root()
print('repo_root =', repo_root)
print('kernel_python =', sys.executable)
if '/.venv/' not in sys.executable.replace('\\', '/'):
    print('WARNING: kernel is not the project .venv interpreter.')

LANGGRAPH_AVAILABLE = importlib.util.find_spec('langgraph') is not None
print('langgraph_available =', LANGGRAPH_AVAILABLE)

## P0 vs P1 (Orchestration Core)

| Concern | Phase 0 (`orchestrator.py`) | Phase 1 (`graph.py`) | Why this matters |
|---|---|---|---|
| Control structure | manual loop with conditionals | explicit graph nodes and edges | clearer execution semantics and extension points |
| State safety | assumes key presence in many spots | `ensure_state_defaults` at node boundaries | resilient against partial/serialized state |
| Duplicate calls | hash tracking in state object | `seen_tool_signatures` guard with retry counter | prevents wasted loops and cost spikes |
| Memoization policy | mostly prompt-driven | explicit policy node + bounded retries | deterministic reliability behavior |

## Why this block exists: inspect graph structure

Inspecting source ensures the compiled graph and node handlers match the intended P1 architecture before runtime tests.

In [ ]:
import inspect
from execution.langgraph.graph import LangGraphOrchestrator

print(inspect.getsource(LangGraphOrchestrator._compile_graph))
print(inspect.getsource(LangGraphOrchestrator._execute_action))

## Why this block exists: offline scripted run

We use scripted model outputs to test orchestration deterministically. This avoids network dependence and isolates control-logic behavior.

In [ ]:
import json
from execution.langgraph.memo_store import SQLiteMemoStore
from execution.langgraph.checkpoint_store import SQLiteCheckpointStore
from execution.langgraph.policy import MemoizationPolicy
from execution.langgraph.graph import LangGraphOrchestrator

class ScriptedProvider:
    def __init__(self, responses):
        self.responses = [json.dumps(item) for item in responses]
        self.index = 0

    def generate(self, messages):
        if self.index < len(self.responses):
            value = self.responses[self.index]
            self.index += 1
            return value
        return self.responses[-1]

if LANGGRAPH_AVAILABLE:
    provider = ScriptedProvider([
        {'action': 'tool', 'tool_name': 'write_file', 'args': {'path': 'fib.txt', 'content': ','.join(str(i) for i in range(150))}},
        {'action': 'tool', 'tool_name': 'memoize', 'args': {'key': 'write_file:fib.txt', 'value': {'ok': True}, 'source_tool': 'write_file'}},
        {'action': 'tool', 'tool_name': 'sort_array', 'args': {'items': [3, 2, 1], 'order': 'asc'}},
        {'action': 'tool', 'tool_name': 'sort_array', 'args': {'items': [3, 2, 1], 'order': 'asc'}},
        {'action': 'finish', 'answer': 'done'}
    ])
    orchestrator = LangGraphOrchestrator(
        provider=provider,
        memo_store=SQLiteMemoStore('.tmp/notebook_demo_graph_memo.db'),
        checkpoint_store=SQLiteCheckpointStore('.tmp/notebook_demo_graph_ckpt.db'),
        policy=MemoizationPolicy(max_policy_retries=2),
        max_steps=30,
    )
    result = orchestrator.run('offline graph demo')
    print(result['answer'])
    print(result['state']['retry_counts'])
    print(result['state']['tool_call_counts'])
else:
    result = None
    print('Install langgraph to run this runtime demo.')

In [ ]:
if LANGGRAPH_AVAILABLE:
    assert result['answer'] == 'done'
    assert result['state']['retry_counts']['duplicate_tool'] >= 1
    assert 'write_file' in result['state']['tool_call_counts']
    print('graph assertions passed')
else:
    print('Skipped assertions because langgraph is not installed in this kernel.')

## Takeaways

- P1 orchestration is graph-native, not only loop-native.
- Duplicate tool call protection and memo policy enforcement are explicit runtime behaviors.
- Node-level checkpoints give recovery and audit leverage absent in P0.
- Next notebook sequence complete. Return to index notebook for cross-links.